In [1]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow..
Next word prediction after n_input words learned from text file.
A story is automatically generated if the predicted word is fed back as input.
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments
'''

from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [2]:
# Read musical data
music_chord = pd.read_csv('../tmp/Debussy_Claire_de_Lune_grades_chords.csv')

In [3]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = '../tmp'
writer = tf.summary.FileWriter(logs_path)

# training_data = read_data(training_file)


training_data = music_chord['grades']
print("Loaded training data...")

def build_dataset(words):
    # A very sad way of creating a dictionary
    count = np.unique(words)
    dictionary = dict()
    for word in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

Loaded training data...


In [4]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [5]:
# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 10

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):
    
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])
    print(x)

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)
    print(x)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    print(rnn_cell)
    
    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    print(outputs)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [6]:
pred = RNN(x, weights, biases)

Tensor("Reshape:0", shape=(?, 10), dtype=float32)
[<tf.Tensor 'split:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:1' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:2' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:3' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:4' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:5' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:6' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:7' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:8' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:9' shape=(?, 1) dtype=float32>]
[<tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_2:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_5:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_8:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_11:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_ce

In [7]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        words = input(prompt)
        print(words)
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(100):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = reverse_dictionary[onehot_pred_index]
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
                print(sentence)
        except:
            print("Word not in dictionary")
    tf.saved_model.simple_save(
            session, '../tmp', symbols_in_keys, symbols_out_onehot
        )

Iter= 1000, Average Loss= 8.337956, Average Accuracy= 0.50%
["('V6', 'III6', 'V3')", "('V6', 'III6', 'I4')", "('V6', 'III6', 'III4')", "('V6', 'V4', 'III6')", "('V6', 'III6', 'I5')", "('V6', 'III6', 'III5')", "('V6', 'V5', 'III6')", "('V6', 'I6', 'III6')", "('V4', 'I4')", "('V4', 'I4', 'I5')"] - [('V4', 'III5', 'I4', 'I5')] vs [('I5', 'X', 'IV5')]
Iter= 2000, Average Loss= 6.141541, Average Accuracy= 1.00%
["('I3', 'I5')", "('III3',)", "('III3', 'VII4')", "('III3', 'III4')", "('III3', 'V4')", "('III3', 'VII5')", "('III3', 'III5')", "('V5', 'V3', 'X')", "('X', 'V5', 'X')", "('V5', 'X', 'X')"] - [('V4', 'V5', 'X')] vs [('VII5', 'III4', 'II5')]
Iter= 3000, Average Loss= 6.013202, Average Accuracy= 2.90%
["('I5', 'III5')", "('X', 'I5', 'IV4', 'III5')", "('X', 'VII5', 'IV4', 'II5')", "('X', 'I5', 'IV4', 'III5')", "('X', 'VII5', 'IV4', 'II5')", "('V4', 'VII5', 'III4', 'II5')", "('V4', 'VI4', 'III4', 'I5')", "('V4', 'VII5', 'III4', 'II5')", "('V4', 'VI4', 'III4', 'I5')", "('V4', 'VI4', 'III4'

Iter= 18000, Average Loss= 2.631342, Average Accuracy= 51.40%
["('X', 'VI4', 'IV4', 'I5')", "('X', 'VI4', 'II4', 'I5')", "('VI3', 'VI4', 'III4', 'I5')", "('VI4', 'X', 'III4', 'I5')", "('II3', 'VI4', 'I4', 'I5')", "('VI2', 'VI4', 'I4', 'I5')", "('II2', 'I4')", "('VI2', 'I4')", "('II3', 'I4')", "('VI3', 'X')"] - [('II3', 'VI3')] vs [('I6', 'I4', 'I5', 'VI4', 'VI5', 'IV4')]
Iter= 19000, Average Loss= 2.323087, Average Accuracy= 57.20%
["('III2', 'V4', 'VII4', 'III5')", "('III2', 'III4', 'VII4', 'III5')", "('II2', 'IV5')", "('II2', 'VI2', 'IV5')", "('II2', 'II3', 'IV5')", "('II2', 'IV3', 'IV5')", "('II2', 'VI3', 'IV5')", "('II2', 'II4', 'IV5')", "('IV4', 'IV5')", "('IV4', 'IV5')"] - [('VI4', 'IV4', 'IV5')] vs [('VI4', 'IV4', 'IV5')]
Iter= 20000, Average Loss= 2.026906, Average Accuracy= 62.40%
["('IV4', 'IV5')", "('IV4', 'IV5')", "('VI4', 'IV4', 'IV5')", "('III4', 'III5')", "('IV4', 'III4', 'III5')", "('VI4', 'III4', 'III5')", "('X', 'X')", "('IV4', 'X', 'X')", "('VI4', 'X', 'X')", "('II4'

Iter= 41000, Average Loss= 1.324693, Average Accuracy= 77.30%
["('V5', 'V4', 'VII5', 'III5')", "('V4', 'VI4', 'V3', 'VII4')", "('I2',)", "('I2', 'V2')", "('I2', 'I3')", "('I2', 'III3')", "('I2', 'V3')", "('I2', 'I4')", "('III2',)", "('III2', 'VII3')"] - [('III2', 'III3')] vs [('III2', 'III3')]
Iter= 42000, Average Loss= 1.210420, Average Accuracy= 77.70%
["('III2', 'VII3')", "('III2', 'III3')", "('III2', 'V3')", "('III2', 'VII4')", "('III2', 'III4')", "('I2',)", "('I2', 'V2')", "('I2', 'I3')", "('I2', 'III3')", "('I2', 'V3')"] - [('I2', 'I4')] vs [('I2', 'I4')]
Iter= 43000, Average Loss= 0.857204, Average Accuracy= 85.20%
["('I2',)", "('I2', 'V2')", "('I2', 'I3')", "('I2', 'III3')", "('I2', 'V3')", "('I2', 'I4')", "('III2',)", "('III2', 'VII3')", "('III2', 'III3')", "('III2', 'V3')"] - [('III2', 'VII4')] vs [('III2', 'VII4')]
Iter= 44000, Average Loss= 1.268074, Average Accuracy= 77.70%
["('I2', 'V2')", "('I2', 'I3')", "('I2', 'III3')", "('I2', 'V3')", "('I2', 'I4')", "('III2',)", "('I

TypeError: 'tuple' object is not callable

In [ ]:
tf.saved_model.simple_save(session,'../tmp/model.ckpt',inputs=dictionary,outputs=dictionary)

In [ ]:
saver = tf.train.Saver()
saver.save(session, "/tmp/model.ckpt")